# Facebook Public Feed Scraper
An implementation of webscraper for **infinate scrolling pages** of Facebook publics feeds<br>


In [ ]:
# !pip install beatifulsoup4
# !pip install getpass
# !pip install selenium

In [1]:
from selenium import webdriver
from getpass import getpass
from bs4 import BeautifulSoup
import time

In [2]:
# set fb url
base_url = 'http://www.facebook.com'

### Setup Web Driver
Specify PATH to driver (download: https://sites.google.com/a/chromium.org/chromedriver/)

In [28]:
PATH = '/Users/christinevinck/anaconda3/envs/SocialWeb/lib/python2.7/site-packages/selenium/webdriver/chrome/chromedriver'
driver = webdriver.Chrome(PATH)

### Login to Facebook

In [29]:
# go to facebook page
driver.get(base_url)

# get email- and password field
email_field = driver.find_element_by_id("email")
password_field = driver.find_element_by_id("pass")

In [30]:
# set login details
email = 'tintilinchi@hotmail.com'
password = getpass()

········


In [31]:
# fill in login details
email_field.send_keys(email)
password_field.send_keys(password)

In [32]:
# login
driver.find_element_by_xpath("//input[@value='Log In']").click()

### Search in Public Stories

In [33]:
# define search term
search_term = 'klm'

# define url for page to scrape
url = base_url + '/search/str/{}/stories-keyword/stories-public'.format(search_term)

# request the page html
driver.get(url)

In [34]:
def getResults(div):
    # go to the results container
    results_div = div.findAll(attrs={"data-testid" : "results"})
    # determine level of results
    results_div = results_div[0] if len(results_div) != 0 else div
    
    # get the resultslen(results_div) != 0 else div.
    results = results_div.findChildren(recursive=False)
    return results

In [35]:
initial_id = 'BrowseResultsContainer'

def getResultsFromContainers(soup):
    # get first container
    initial_container = soup.find(id=initial_id)
    
    # if not already retrieved
    if initial_id not in container_results.keys():
        # get container id
        container_id = initial_container.attrs['id']
        # retrieve results from container
        yield container_id, getResults(initial_container)

    # get remaining containers (except last)
    for container in initial_container.find_next_siblings()[:-1]:
        # get container id
        container_id = container.attrs['id']  
        # if not already retrieved
        if container_id not in container_results.keys():
            # retrieve results from container
            yield container_id, getResults(container)

### Scrape (finite) scrolling 
TODO: use while loop instead of for loop

In [36]:
container_results = {}

In [37]:
def scrapeScrollingPage(n):

    for i in range(0, n):
        print('Scraping scrolling: {}'.format(i))

        # scroll to the end of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(10)

        # retrieve html
        html_source = driver.page_source
        data = html_source.encode('utf-8')

        # create soup
        soup = BeautifulSoup(data, 'html.parser')
        # parse to get data
        for container_id, results in getResultsFromContainers(soup):
            # store results
            print('Container: {} '.format(container_id) + 'has {} results'.format(len(results)))
            container_results[container_id] = results

In [38]:
# Scrape for 40 times scrolling (TODO: use while...)
n = 40
scrapeScrollingPage(n)

Scraping scrolling: 0
Container: BrowseResultsContainer has 2 results
Container: u_ps_0_3_0_browse_result_below_fold has 3 results
Container: fbBrowseScrollingPagerContainer0 has 3 results
Container: fbBrowseScrollingPagerContainer1 has 3 results
Container: fbBrowseScrollingPagerContainer2 has 3 results
Scraping scrolling: 1
Container: fbBrowseScrollingPagerContainer3 has 3 results
Container: fbBrowseScrollingPagerContainer4 has 3 results
Scraping scrolling: 2
Container: fbBrowseScrollingPagerContainer5 has 3 results
Container: fbBrowseScrollingPagerContainer6 has 3 results
Scraping scrolling: 3
Container: fbBrowseScrollingPagerContainer7 has 3 results
Container: fbBrowseScrollingPagerContainer8 has 3 results
Scraping scrolling: 4
Container: fbBrowseScrollingPagerContainer9 has 2 results
Container: fbBrowseScrollingPagerContainer10 has 3 results
Scraping scrolling: 5
Container: fbBrowseScrollingPagerContainer11 has 3 results
Container: fbBrowseScrollingPagerContainer12 has 3 results
Sc

**NOTE:** After 17 scrolls, this page reaches -- End of Results --, which is in no means all the results to be generated, so we should think about how to cope with this.
![title](img/end-of-results.png)

### Parse results to obtain raw data

In [40]:
# Parse Results (TODO: parse before storing)
total_results_count = 0

for key, results in container_results.iteritems():
    total_results_count += len(results)

    for div in results:
        # get header
        header_divs = div.find('div', attrs={'class': 'clearfix'})\
            .findAll('div', recursive=False)[0]\
            .findAll('a')
        
        # get company name
        name_div = header_divs[0]
        name = name_div.get_text()
        print(name)
        
        # get date
        date_div = header_divs[2]
        date = date_div.get_text()
        print(date)

        # TODO:get verified?   
        
        # TODO: get loves/likes/sad, comments, shares
    
total_results_count

Airplane-Pictures.net
March 3, 2016 at 8:31pm
Business Today
7 hours ago
KLM Royal Dutch Airlines
March 11, 2013 at 4:31pm
Aeroflap
February 21, 2017 at 4:57pm
Mazhavil Manorama
September 8, 2017 at 1:16pm
BoardingPass
52 minutes ago
KLM Royal Dutch Airlines
February 26 at 12:48pm
KLM Royal Dutch Airlines
March 10, 2015 at 10:11am
O.R. Tambo International Airport
February 17, 2017 at 12:38pm
KLM Royal Dutch Airlines
March 6, 2014 at 1:53pm
KLM Royal Dutch Airlines
January 25, 2016 at 4:00pm
Hitradio Namibia
36 new photos
KLM Royal Dutch Airlines
June 8, 2017 at 4:49pm
Secret Flying
March 2 at 2:39pm
KLM Royal Dutch Airlines
October 31, 2016 at 10:58am
KLM Royal Dutch Airlines
March 7, 2016 at 10:00am
Ramsay World Travel, Kirkcaldy
March 1 at 1:04pm
Secret Flying
December 21, 2016 at 2:57pm
KLM Royal Dutch Airlines
September 29, 2014 at 4:26pm
Grumpy Aliens Korea
January 9 at 9:22am
Mazhavil Manorama
November 24, 2017 at 2:00pm
A Fly Guy's Cabin Crew Lounge
November 1, 2012 at 4:52pm
Am

103

### Search by year

In [41]:
start_year = '2017'
end_year = start_year

query_filter_by_year = '?filters_rp_creation_time=%7B"name"%3A"creation_time"%2C"args"%3A"%7B%5C"start_year%5C"%3A%5C"{}%5C"%2C%5C"start_month%5C"%3A%5C"{}-01%5C"%2C%5C"end_year%5C"%3A%5C"{}%5C"%2C%5C"end_month%5C"%3A%5C"{}-12%5C"%7D"%7D'\
                        .format(start_year, start_year, end_year, end_year)

In [42]:
query_url = url + query_filter_by_year
# request the page html
driver.get(query_url)

In [44]:
container_results = {}
scrapeScrollingPage(n)

Scraping scrolling: 0


error: [Errno 54] Connection reset by peer